NPR Stations DataFrame by County
=========
---------------
Create a Dataframe including following columns:
* FIPS Code
* County Name
* State
* City
* Station Identifier
* AM/FM
* Frequency
* Twitter Account

## Import Libraries and configuration files


In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.offline as py
import requests
import json
from config import gkey

In [30]:
#Read CSV and create dataframe
npr_stations_csv = "Public_Radio_Twitter_Handles.csv"
npr_stations_df = pd.read_csv(npr_stations_csv)
npr_stations_df.dtypes

Index                   int64
State                  object
City                   object
Wiki_name              object
Station Identifier     object
AM/FM                  object
Channel Number        float64
Twitter Handle         object
Alternate              object
Twitter                object
_merge                 object
Empty_at_load          object
dtype: object

In [31]:
npr_stations_df.head()

,Index,State,City,Wiki_name,Station Identifier,AM/FM,Channel Number,Twitter Handle,Alternate,Twitter,_merge,Empty_at_load
0,702,Alaska,NaN,Anchorage,KNBA,FM,90.3,@KNBA,@aprn,Twitter,right_only,Emptyatload
1,703,Alaska,NaN,Juneau,KRNN,FM,102.7,None,NaN,Twitter,right_only,Emptyatload
2,704,Alaska,NaN,Juneau,KXLL,FM,100.7,None,NaN,Twitter,right_only,Emptyatload
3,705,Alaska,NaN,Kotzebue,KOTZ,AM,720.0,None,NaN,Twitter,right_only,Emptyatload
4,706,Alaska,NaN,Sand Point,KSDP,AM,830.0,None,NaN,Twitter,right_only,Emptyatload


In [38]:
npr_stations_df = npr_stations_df.loc[npr_stations_df['Twitter Handle'] != 'None']
npr_stations_df.head()

,Index,State,City,Wiki_name,Station Identifier,AM/FM,Channel Number,Twitter Handle,Alternate,Twitter,_merge,Empty_at_load
0,702,Alaska,NaN,Anchorage,KNBA,FM,90.3,@KNBA,@aprn,Twitter,right_only,Emptyatload
6,708,Alaska,NaN,Unalaska,KUCB,FM,89.7,@KUCBNews,NaN,Twitter,right_only,Emptyatload
10,712,California,NaN,Angwin,KDFC,FM,89.9,@ClassicalKDFC,NaN,Twitter,right_only,Emptyatload
12,714,California,NaN,Calexico,KQVO,FM,97.7,@kpbsnews,NaN,Twitter,right_only,Emptyatload
13,715,California,NaN,Coachella,KVLA,FM,90.3,@kvlatv,NaN,Twitter,right_only,Emptyatload


In [98]:
for (idx, row) in npr_stations_df.iterrows():
    city = (row.loc['City'])
    wiki = (row.loc['Wiki_name'])
    if (pd.isnull(city)):
        npr_stations_df.at[idx, 'City'] = wiki


In [101]:
npr_stations_df.head()

,Index,State,City,Wiki_name,Station Identifier,AM/FM,Channel Number,Twitter Handle,Alternate,Twitter,_merge,Empty_at_load
0,702,Alaska,Anchorage,Anchorage,KNBA,FM,90.3,@KNBA,@aprn,Twitter,right_only,Emptyatload
6,708,Alaska,Unalaska,Unalaska,KUCB,FM,89.7,@KUCBNews,NaN,Twitter,right_only,Emptyatload
10,712,California,Angwin,Angwin,KDFC,FM,89.9,@ClassicalKDFC,NaN,Twitter,right_only,Emptyatload
12,714,California,Calexico,Calexico,KQVO,FM,97.7,@kpbsnews,NaN,Twitter,right_only,Emptyatload
13,715,California,Coachella,Coachella,KVLA,FM,90.3,@kvlatv,NaN,Twitter,right_only,Emptyatload


In [102]:
#Save DataFrame to CSV File (Checkpoint)
npr_stations_df.to_csv('npr_stations_cities_in_one_column.csv')

In [103]:
npr_stations_df = npr_stations_df.drop(['Empty_at_load'], axis=1)
npr_stations_df = npr_stations_df.drop(['Index'], axis=1)
npr_stations_df = npr_stations_df.drop(['Wiki_name'], axis=1)
npr_stations_df = npr_stations_df.drop(['_merge'], axis=1)
npr_stations_df = npr_stations_df.drop(['Twitter'], axis=1)

In [106]:
npr_stations_df.head()


,State,City,Station Identifier,AM/FM,Channel Number,Twitter Handle,Alternate
0,Alaska,Anchorage,KNBA,FM,90.3,@KNBA,@aprn
6,Alaska,Unalaska,KUCB,FM,89.7,@KUCBNews,NaN
10,California,Angwin,KDFC,FM,89.9,@ClassicalKDFC,NaN
12,California,Calexico,KQVO,FM,97.7,@kpbsnews,NaN
13,California,Coachella,KVLA,FM,90.3,@kvlatv,NaN


In [107]:
#Variables to be used during data collection
processed = 0
notfound = 0

#Loop to collect Lat/Long for Cities with NPR Stations
for (idx, row) in npr_stations_df.iterrows():
    try:
        #Store current row value for each column
        city = (row.loc['City'])
        state = (row.loc['State'])
        #URL from Google APIs where Lat/Long values will be collected
        query_url = "https://maps.googleapis.com/maps/api/geocode/json?address="+str(city)+","+str(state)+"&key="+str(gkey)
        #JSON request
        response = requests.get(query_url).json()
        #Variables to store lat/long values 
        lat = response["results"][0]["geometry"]["location"]["lat"]
        lng = response["results"][0]["geometry"]["location"]["lng"]
        #Lat/Long columns created and respective values stored in current row
        npr_stations_df.at[idx, 'Latitude'] = lat
        npr_stations_df.at[idx, 'Longitude'] = lng
        #Print to verify data is processing as expected
        print(str(city)+","+str(state)+":"+str(lat)+","+str(lng))
        #Count to validate how many cities have been processed
        processed += 1
    #Error handling if some of the cities are not found
    except:
        #Print to verify when a city is not found
        print("City not found")
        #Count to validate how many cities were not found
        notfound += 1
        continue
#Print to validate final count for both processed and not found cities
print(str(processed)+" cities were processed. "+str(notfound)+" cities were not found.")

Anchorage,Alaska:61.2180556,-149.9002778
Unalaska,Alaska:53.8844382,-166.5332434
Angwin,California:38.5757402,-122.4499796
Calexico,California:32.6789476,-115.4988834
Coachella,California:33.6803003,-116.173894
McCloud,California:41.2557119,-122.1394487
Mojave,California:35.0524699,-118.1739645
Morro Bay,California:35.3659445,-120.8499924
Quincy,California:39.9368363,-120.9471764
Redlands,California:34.0555693,-117.1825381
Sacramento,California:38.5815719,-121.4943996
San Ardo,California:36.0205207,-120.9051826
San Francisco,California:37.7749295,-122.4194155
Santa Barbara,California:34.4208305,-119.6981901
Santa Barbara,California:34.4208305,-119.6981901
Santa Barbara,California:34.4208305,-119.6981901
Sutter,California:39.1598915,-121.7527482
Thousand Oaks,California:34.1705609,-118.8375937
Carbondale,Colorado:39.4022054,-107.2111628
Carbondale,Colorado:39.4022054,-107.2111628
Denver,Colorado:39.7392358,-104.990251
Denver,Colorado:39.7392358,-104.990251
Fort Collins,Colorado:40.58526

Spindale,North Carolina:35.3601212,-81.929271
Wilmington,North Carolina:34.2257255,-77.9447102
Winston-Salem,North Carolina:36.09985959999999,-80.244216
Kingston,North Carolina:35.2626635,-77.5816353
Rocky Mount,North Carolina:35.9382103,-77.7905339
Elizabeth City,North Carolina:36.2946008,-76.2510461
Kinston,North Carolina:35.2626635,-77.5816353
Manteo,North Carolina:35.9082256,-75.67573019999999
New Bern,North Carolina:35.108493,-77.04411429999999
Raleigh,North Carolina:35.7795897,-78.6381787
Raleigh,North Carolina:35.7795897,-78.6381787
Rocky Mount,North Carolina:35.9382103,-77.7905339
Winston-Salem,North Carolina:36.09985959999999,-80.244216
Belcourt,North Dakota:48.8391712,-99.7448689
Bismarck,North Dakota:46.8083268,-100.7837392
Dickinson,North Dakota:46.8791756,-102.7896242
Fargo,North Dakota:46.8771863,-96.7898034
Grand Forks,North Dakota:47.9252568,-97.0328547
Jamestown,North Dakota:46.9105438,-98.7084357
Williston,North Dakota:48.1469683,-103.6179745
Athens,Ohio:39.3292396,-8

Roanoke,Virginia:37.2709704,-79.9414266
Columbus,Ohio:39.9611755,-82.99879419999999
Cleveland,Ohio:41.49932,-81.6943605
Norfolk,Virginia:36.8507689,-76.28587259999999
Allentown,Pennsylvania:40.6022939,-75.4714098
Johnson City,Tennessee:36.3134397,-82.3534727
Marion,Virginia:36.8348383,-81.5148368
Norfolk,Virginia:36.8507689,-76.28587259999999
Philadelphia,Pennsylvania:39.9525839,-75.1652215
Harrisburg,Pennsylvania:40.2731911,-76.8867008
Memphis,Tennessee:35.1495343,-90.0489801
Kent,Ohio:41.1536674,-81.3578859
St. Thomas,US Virgin Islands:18.3380965,-64.8940946
Christiansburg,Virginia:37.1298517,-80.4089389
Murfreesboro,Tennessee:35.8456213,-86.39027
Farmville,Virginia:37.3020965,-78.3919401
San Juan,Puerto Rico:18.4655394,-66.1057355
Columbus,Ohio:39.9611755,-82.99879419999999
Cambridge,Ohio:40.031183,-81.5884561
Madison,Tennessee:36.25620019999999,-86.7143516
Nashville,Tennessee:36.1626638,-86.7816016
State College,Pennsylvania:40.7933949,-77.8600012
Johnstown,Pennsylvania:40.32674069

Rochester,Minnesota:44.0121221,-92.4801989
St. Paul,Minnesota:44.9537029,-93.0899578
St. Peters,Minnesota:44.3258932,-93.955691
Worthington,Minnesota:43.6210985,-95.59414360000001
Northfield,Minnesota:44.4582983,-93.161604
Rochester,Minnesota:44.0121221,-92.4801989
Morehead,Kentucky:38.1839705,-83.4326841
Helena,Montana:46.5891452,-112.0391057
Kalispell,Montana:48.1919889,-114.316813
Missoula,Montana:46.8721284,-113.9940314
Albany,Georgia:31.5785074,-84.15574099999999
Brunswick,Georgia:31.1499528,-81.49148939999999
Dahlonega,Georgia:34.5261465,-83.9843953
Demorest,Georgia:34.565098,-83.5451724
Ft. Gaines,Georgia:31.6090569,-85.0471505
Valdosta,Georgia:30.8327022,-83.2784851
Waycross,Georgia:31.2135511,-82.3540178
Macon,Georgia:32.8406946,-83.6324022
Hastings,Nebraska:40.5862583,-98.38987259999999
Springfield,Illinois:39.78172130000001,-89.6501481
Chico,California:39.7284944,-121.8374777
Redding,California:40.5865396,-122.3916754
Moscow,Idaho:46.73238749999999,-117.000165
Kenai,Alaska:6

In [108]:
npr_stations_df.head(10)

,State,City,Station Identifier,AM/FM,Channel Number,Twitter Handle,Alternate,Latitude,Longitude
886,Florida,Tampa,WUSF,FM,89.7,@wusf,NaN,27.950575,-82.457178
887,Florida,Pensacola,WUWF,FM,88.1,@wuwf,NaN,30.421309,-87.216915
888,Alabama,Montgomery,WVAS,FM,90.7,@WVAS,NaN,32.366805,-86.299969
889,Illinois,Rock Island,WVIK,FM,90.3,@wvikfm,NaN,41.509477,-90.578748
890,Indiana,Elkhart,WVPE,FM,88.1,@wvpe,NaN,41.681993,-85.976667
891,Louisana,New Orleans,WWNO,FM,89.9,@wwno,NaN,29.951066,-90.071532
892,Louisana,Thibodaux,KTLN,FM,90.5,@wwno,NaN,29.795763,-90.822871
893,Florida,West Palm Beach,WXEL,FM,90.7,@wxeltv,NaN,26.715342,-80.053375
894,Maryland,Baltimore,WYPR,FM,88.1,@wypr881FM,NaN,39.290385,-76.612189
895,Maryland,Frostburg,WFWM,FM,91.9,@xfsrfrostburg,NaN,39.658142,-78.928357


In [109]:
#Save DataFrame to CSV File
npr_stations_df.to_csv('npr_stations_by_city.csv')

In [110]:
#Read CSV and create Dataframe
npr_by_county_csv = 'npr_stations_by_city.csv'
npr_by_county_df = pd.read_csv(npr_by_county_csv)

#URL to get FIPS code
url = 'https://geo.fcc.gov/api/census/block/find?'
#Variables to be used during data collection
processed = 0
notfound = 0

#Loop to add FIPS code and County name to Dataframe
for (idx, row) in npr_by_county_df.iterrows():
    try:
        #Store current row value for each column
        latitude = (row.loc['Latitude'])
        longitude = (row.loc['Longitude'])
        #URL to collect FIPS code and county name
        query_url = str(url)+"&latitude="+str(latitude)+"&longitude="+str(longitude)+"&format=json"
        #JSON Request
        code = requests.get(query_url).json()
        #Variables to store required values (FIPS codes are converted to strings to keep leading zeros in the code)
        county_fips = str(code["County"]["FIPS"]).zfill(5)
        county_name = code["County"]["name"]
        block_fips = str(code["Block"]["FIPS"]).zfill(15)
        #New columns created and values collected from JSON stored in current row 
        npr_by_county_df.at[idx, 'FIPS'] = str(county_fips)
        npr_by_county_df.at[idx, 'County Name'] = county_name
        npr_by_county_df.at[idx, 'FIPS_block'] = str(block_fips)         
        #Count to validate how many records were processed
        processed += 1
        #Print to verify that information is processed
        print(str(county_name)+":"+str(county_fips))
    #Error handling if a location is not found
    except:
        #Print to validate if a location is not found
        print("Location Not Found")
        #Count to validate how many records were not found
        notfound += 1
        pass
#Print to validate final count for both processed and not found locations
print(str(processed)+" locations were processed. "+str(notfound)+" locations were not found.")


Anchorage:02020
Aleutians West:02016
Napa:06055
Imperial:06025
Riverside:06065
Siskiyou:06093
Kern:06029
San Luis Obispo:06079
Plumas:06063
San Bernardino:06071
Sacramento:06067
Monterey:06053
San Francisco:06075
Santa Barbara:06083
Santa Barbara:06083
Santa Barbara:06083
Sutter:06101
Ventura:06111
Garfield:08045
Garfield:08045
Denver:08031
Denver:08031
Larimer:08069
La Plata:08067
Routt:08107
Fairfield:09001
New Haven:09009
Middlesex:09007
Litchfield:09005
Kent:10001
Miami-Dade:12086
Sarasota:12115
Murray:13213
Bibb:13021
Floyd:13115
Knox:17095
DuPage:17043
Champaign:17019
Allen:18003
Lake:18089
Knox:18083
Dickinson:19059
Wapello:19179
Wapello:19179
Madison:19121
Marion:19125
Black Hawk:19013
Neosho:20133
Ellis:20051
Pottawatomie:20149
Jefferson:21111
Rapides:22079
East Baton Rouge:22033
Tangipahoa:22105
Lafayette:22055
Ouachita:22073
Orleans:22071
Caddo:22017
Knox:23013
Frederick:24021
Washington:24043
Baltimore:24005
Barnstable:25001
Franklin:25011
Middlesex:25017
Essex:25009
Middle

Nicollet:27103
Nobles:27105
Rice:27131
Olmsted:27109
Rowan:21205
Lewis and Clark:30049
Flathead:30029
Missoula:30063
Dougherty:13095
Glynn:13127
Lumpkin:13187
Habersham:13137
Clay:13061
Lowndes:13185
Ware:13299
Bibb:13021
Adams:31001
Sangamon:17167
Butte:06007
Shasta:06089
Latah:16057
Kenai Peninsula:02122
Reno:20155
Saline:20169
Sitka:02220
Rapides:22079
Caddo:22017
Boundary:16021
East Baton Rouge:22033
Pennington:27113
LaSalle:17099
McDonough:17109
Houston:01069
Montgomery:01101
Muscogee:13215
Fulton:13121
Berkshire:25003
District of Columbia:11001
Tippecanoe:18157
Tippecanoe:18157
Cook:17031
Jefferson:01073
Etowah:01055
Allen:18003
Steuben:18151
Suffolk:25025
Barnstable:25001
Barnstable:25001
Peoria:17143
Alpena:26007
Emmet:26047
Isabella:26073
Chippewa:26033
Wayne:26163
Honolulu:15003
Honolulu:15003
Windham:09015
Washtenaw:26161
District of Columbia:11001
Hampshire:25015
Champaign:17019
Brevard:12009
Monroe:18105
Jefferson:21111
Jefferson:21111
Bay:12005
Leon:12073
Leon:12073
Mario

In [113]:
#Show dataframe
npr_by_county_df.head(10)

,Unnamed: 0,State,City,Station Identifier,AM/FM,Channel Number,Twitter Handle,Alternate,Latitude,Longitude,FIPS,County Name,FIPS_block
0,0,Alaska,Anchorage,KNBA,FM,90.3,@KNBA,@aprn,61.218056,-149.900278,02020,Anchorage,020200011001015
1,6,Alaska,Unalaska,KUCB,FM,89.7,@KUCBNews,NaN,53.884438,-166.533243,02016,Aleutians West,020160002001050
2,10,California,Angwin,KDFC,FM,89.9,@ClassicalKDFC,NaN,38.575740,-122.449980,06055,Napa,060552017003000
3,12,California,Calexico,KQVO,FM,97.7,@kpbsnews,NaN,32.678948,-115.498883,06025,Imperial,060250121001006
4,13,California,Coachella,KVLA,FM,90.3,@kvlatv,NaN,33.680300,-116.173894,06065,Riverside,060650457071006
5,14,California,McCloud,KLDD,FM,91.9,@JPRnews,NaN,41.255712,-122.139449,06093,Siskiyou,060930012001686
6,16,California,Mojave,KCRY,FM,88.1,@kcrw,NaN,35.052470,-118.173964,06029,Kern,060290059002066
7,17,California,Morro Bay,KESC,FM,99.7,@classicalkusc,NaN,35.365944,-120.849992,06079,San Luis Obispo,060790106031052
8,20,California,Quincy,KQNC,FM,88.1,@capradionews,NaN,39.936836,-120.947176,06063,Plumas,060630001001050
9,21,California,Redlands,KUOR,FM,89.1,@scpr,NaN,34.055569,-117.182538,06071,San Bernardino,060710081001016


In [114]:
#Drop rows with no FIPS 
npr_by_county_df = npr_by_county_df[npr_by_county_df['FIPS'].notnull()]

#Loop to fill leading zeros on FIPS codes
for (idx, row) in npr_by_county_df.iterrows():
    fips_z = (row.loc['FIPS'])
    fips_block_z = (row.loc['FIPS_block'])
    npr_by_county_df.at[idx, 'FIPS'] = str(fips_z).zfill(5)
    npr_by_county_df.at[idx, 'FIPS_block'] = str(fips_block_z).zfill(15)

In [115]:
#Show row count group by FIPS code
count_fips = npr_by_county_df[['City','FIPS']]
count_fips = count_fips.groupby('FIPS').count()
count_fips = count_fips.rename(columns={'City':'Count'})
count_fips = count_fips.sort_values(by=['Count'], ascending=[False])
count_fips.head(10)


,Count
FIPS,
06037,8
25001,6
41029,5
27137,5
08031,5
27109,4
09001,4
06083,4
36029,4


In [116]:
#Reorder dataframe columns
npr_by_county_df = npr_by_county_df[["FIPS", "County Name", "City",  "State", "Latitude", "Longitude", "Station Identifier", "Channel Number", "AM/FM","Twitter Handle","Alternate","FIPS_block"]]
#Sort records by FIPS code
npr_by_county_df = npr_by_county_df.sort_values(by=['FIPS'])

#Show dataframe
npr_by_county_df.head(20)

,FIPS,County Name,City,State,Latitude,Longitude,Station Identifier,Channel Number,AM/FM,Twitter Handle,Alternate,FIPS_block
651,01015,Calhoun,Jacksonville,Alabama,33.813713,-85.761354,WLJS,91.9,FM,@WLJS_919,NaN,010150021012089
412,01033,Colbert,Muscle Shoals,Alabama,34.744811,-87.667529,WQPR,88.7,FM,@Alpublicradio,NaN,010330207042005
413,01047,Dallas,Selma,Alabama,32.407359,-87.021101,WAPR,88.3,FM,@Alpublicradio,NaN,010479566003019
603,01055,Etowah,Gadsden,Alabama,34.014264,-86.006639,WSGN,91.5,FM,@WBHM903,NaN,010550012001034
593,01069,Houston,Dothan,Alabama,31.223231,-85.390489,WRWA,88.7,FM,@troypublicradio,NaN,010690406001032
602,01073,Jefferson,Birmingham,Alabama,33.520661,-86.802490,WBHM,90.3,FM,@WBHM,NaN,010730027001097
639,01089,Madison,Huntsville,Alabama,34.730369,-86.586104,WJAB,90.9,FM,@WJAB-TV,NaN,010890031001024
652,01089,Madison,Huntsville,Alabama,34.730369,-86.586104,WLRH,89.3,FM,@WLRH_893FMHD,NaN,010890031001024
635,01097,Mobile,Mobile,Alabama,30.695366,-88.039891,WHIL,91.3,FM,@WHILFM,NaN,010970012001266
694,01101,Montgomery,Montgomery,Alabama,32.366805,-86.299969,WVAS,90.7,FM,@WVAS,NaN,011010015003009


In [117]:
#Save DataFrame to CSV File
npr_stations_df.to_csv('npr_stations_twitter_by_county.csv')